# Необходимые библиотеки

In [ ]:
import numpy as np
from sklearn.datasets import load_diabetes
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis # LDA
from sklearn.preprocessing import StandardScaler # вычисляет среднее и стандартное отклонение для каждого признака
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_squared_log_error, mean_absolute_error, r2_score, silhouette_score # метрики
import time
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.cluster import KMeans
import csv
import requests
from bs4 import BeautifulSoup
from sklearn.decomposition import PCA
from sklearn.feature_extraction.text import TfidfTransformer, TfidfVectorizer

# Линейная регрессия

*Задание 1. Используйте датасет sklearn.datasets.load_diabetes(). Разобраться с тем, какие данные в нём содержатся, а также какая переменная является целевой, можно по ссылке;*

In [ ]:

# Загрузка датасета
diabetes = load_diabetes()

# Информация о датасете
print("Описание датасета:")
print(diabetes.DESCR)
print()


Описание датасета:
.. _diabetes_dataset:

Diabetes dataset
----------------

Ten baseline variables, age, sex, body mass index, average blood
pressure, and six blood serum measurements were obtained for each of n =
442 diabetes patients, as well as the response of interest, a
quantitative measure of disease progression one year after baseline.

**Data Set Characteristics:**

  :Number of Instances: 442

  :Number of Attributes: First 10 columns are numeric predictive values

  :Target: Column 11 is a quantitative measure of disease progression one year after baseline

  :Attribute Information:
      - age     age in years
      - sex
      - bmi     body mass index
      - bp      average blood pressure
      - s1      tc, total serum cholesterol
      - s2      ldl, low-density lipoproteins
      - s3      hdl, high-density lipoproteins
      - s4      tch, total cholesterol / HDL
      - s5      ltg, possibly log of serum triglycerides level
      - s6      glu, blood sugar level

No

In [ ]:
# Вывод названий признаков
print("Названия признаков:")
print(diabetes.feature_names)
print()


Названия признаков:
['age', 'sex', 'bmi', 'bp', 's1', 's2', 's3', 's4', 's5', 's6']



In [ ]:
# Вывод размерности данных
print("Размерность данных:")
print(diabetes.data.shape)
print()


Размерность данных:
(442, 10)



In [ ]:
# Вывод размерности целевой переменной
print("Размерность целевой переменной:")
print(diabetes.target.shape)

Размерность целевой переменной:
(442,)


*Задание 2. Используйте любой известный алгоритм понижения размерности (например, LDA) для того, чтобы снизить количество признаков до одного (вариант примитивнее – взять любую переменную исходного датасета, которую Вы считаете наиболее значимой).*

In [ ]:
X = diabetes.data
y = diabetes.target

# Инициализация и обучение LDA с одной компонентой
lda = LinearDiscriminantAnalysis(n_components=1)
X_lda = lda.fit_transform(X, y)

In [ ]:
print("Размерность данных после LDA:")
print(X_lda.shape)

Размерность данных после LDA:
(442, 1)


*Задание 3. Реализуйте алгоритм линейной регрессии с использованием градиентного спуска и функциями потерь SMSE (см. выше) и SMAE (продифференцируйте самостоятельно). Обратите внимание, что для данного пункта запрещается использовать готовые реализации методов (LinearRegression, mean_squared_error и т.д.);*

In [ ]:

def linear_regression_gd(X, y, learning_rate=0.01, n_iterations=100):
    n_samples, n_features = X.shape
    weights = np.zeros(n_features)
    bias = 0

    for _ in range(n_iterations):
        # Градиенты функций потерь по весам и смещению
        predictions = np.dot(X, weights) + bias
        d_weights = (2/n_samples) * np.dot(X.T, (predictions - y))
        d_bias = (2/n_samples) * np.sum(predictions - y)

        # Обновление весов и смещения с помощью градиентного спуска
        weights -= learning_rate * d_weights
        bias -= learning_rate * d_bias

    return weights, bias

def smse_loss(X, y, weights, bias):
    n_samples = len(y)
    predictions = np.dot(X, weights) + bias
    return np.sum((predictions - y) ** 2) / n_samples # среднее значение квадратов ошибок

def smae_loss(X, y, weights, bias):
    n_samples = len(y)
    predictions = np.dot(X, weights) + bias
    return np.sum(np.abs(predictions - y)) / n_samples

In [ ]:
# Стандартизация
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Обучение модели
weights, bias = linear_regression_gd(X_scaled, y, learning_rate=0.01, n_iterations=100)

# Предсказание и расчет потерь
predictions = np.dot(X_scaled, weights) + bias
smse_loss_value = smse_loss(X_scaled, y, weights, bias)
smae_loss_value = smae_loss(X_scaled, y, weights, bias)

print("Weights:", weights)
print("Bias:", bias)
print("SMSE Loss:", smse_loss_value)
print("SMAE Loss:", smae_loss_value)

Weights: [ 0.47843451 -8.88693544 22.66362833 14.15231087 -1.7709999  -4.79357878
 -9.81312873  6.17531372 18.87619539  5.84479904]
Bias: 131.95760905649126
SMSE Loss: 3315.066530264702
SMAE Loss: 46.039385491848336


*Задание 4. Теперь постройте прогнозы, используя стандартную реализацию LinearRegression из sklearn;*

In [ ]:
# Разбивка на обуч. и тест. выборку
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Инициализация и обучение модели линейной регрессии
model = LinearRegression()
model.fit(X_train, y_train)

# Прогнозы на тестовом наборе
predictions = model.predict(X_test)

# Оценка качества модели
mse = mean_squared_error(y_test, predictions)
mae = mean_absolute_error(y_test, predictions)

print("Mean Squared Error (MSE):", mse)
print("Mean Absolute Error (MAE):", mae)

Mean Squared Error (MSE): 2900.193628493481
Mean Absolute Error (MAE): 42.79409467959994


*Задание 5. Сравните основные метрики качества для "собственной"реализации и
варианта из sklearn – MSE, MSLE, MAE, R2, RMSE. Какой из двух алго-
ритмов оказался эффективнее? Какой менее подвержен переобучению?*

In [ ]:
# собственные функции:


def mse(y_true, y_pred):
    return np.mean((y_true - y_pred) ** 2)

def msle(y_true, y_pred):
    return np.mean((np.log1p(y_true) - np.log1p(y_pred)) ** 2)

def mae(y_true, y_pred):
    return np.mean(np.abs(y_true - y_pred))

def r2(y_true, y_pred):
    ss_res = np.sum((y_true - y_pred) ** 2)
    ss_tot = np.sum((y_true - np.mean(y_true)) ** 2)
    return 1 - (ss_res / ss_tot)

def rmse(y_true, y_pred):
    return np.sqrt(mse(y_true, y_pred))

In [ ]:
# Обучение модели линейной регрессии из scikit-learn
model_sklearn = LinearRegression()
model_sklearn.fit(X_train, y_train)
y_pred_sklearn = model_sklearn.predict(X_test)

In [ ]:
print("Метрики для реализации из scikit-learn:")
print("MSE:", mean_squared_error(y_test, y_pred_sklearn))
print("MSLE:", mean_squared_log_error(y_test, y_pred_sklearn))
print("MAE:", mean_absolute_error(y_test, y_pred_sklearn))
print("R2:", r2_score(y_test, y_pred_sklearn))
print("RMSE:", np.sqrt(mean_squared_error(y_test, y_pred_sklearn)))

Метрики для реализации из scikit-learn:
MSE: 2900.193628493481
MSLE: 0.17696674535345117
MAE: 42.79409467959994
R2: 0.4526027629719196
RMSE: 53.85344583676592


In [ ]:
# Обучение собственной реализации и расчет предсказаний
weights, bias = linear_regression_gd(X_train, y_train, learning_rate=0.01, n_iterations=100)
y_pred_custom = np.dot(X_test, weights) + bias

In [ ]:
# Метрики (значения) для собственной реализации
print("\nМетрики для собственной реализации:")
print("MSE:", mse(y_test, y_pred_custom))
print("MSLE:", msle(y_test, y_pred_custom))
print("MAE:", mae(y_test, y_pred_custom))
print("R2:", r2(y_test, y_pred_custom))
print("RMSE:", rmse(y_test, y_pred_custom))


Метрики для собственной реализации:
MSE: 3477.7013963638624
MSLE: 0.22174820403793408
MAE: 47.322117819763434
R2: 0.34360102136795834
RMSE: 58.97203910637534


**Вывод:** *Сравнив значение метрик можно сказать, что собственная реализация выигрывает для MSE и RMSE. А для метрик таких как R2, MSLE и MAE - выгодней использовать реализованные в scikit-learn при кол-во итераций = 100. Если же поставить = 1000, то значение метрики MSLE в собственной будет ниже, чем во вллженной библиотеке scikit-learn*

*Задание 6. Постройте на плоскости графики прямых (регрессий) для "собствен-
ной"реализации и варианта из sklearn.*

# Кластеризация: о вкусах не спорят

In [ ]:
import os
!pip install yandex_music
from yandex_music import Client

# при корректной работе необходима авторизация! - токен не работает
TOKEN = os.environ.get('y0_AgAEA7qkKEIPAAG8XgAAAAD-tBPcAADRUVD4S2NCvr0j9hN0Y7E6vTDBOA')

client = Client(TOKEN).init()

set_music = {13247: 0, # paramore
             1055688: 0, # paramore
             43233: 1, # skillet
             1170182: 1, # skillet
             488052: 2, # linkin park
             154525: 2, # linkin park
             3579166: 3, # adele
             718991: 3, # adele
             7112: 4, # Rihanna
             2482211: 4, # Rihanna
             }

df = pd.DataFrame(columns=['text','label'])

for i in set_music.keys():
  album = client.albums_with_tracks(i)
  for volume in album.volumes:
    for track in volume:
      try:
        text = track.get_lyrics().fetch_lyrics().lower()
        # почистим от знаков препинания
        text = "".join(c for c in text if c not in ('!','.',':', ',', '?', '—', '(', ')'))
        text = text.replace('\n',' ')
        df.loc[ len(df.index)] = [text, musician[i]]
      except:
        text

NetworkError: {'name': 'Unavailable For Legal Reasons', 'message': ''}  (451): b'{"invocationInfo":{"req-id":"1710814932522206-1025892319874328715","hostname":"music-stable-back-vla-73.vla.yp-c.yandex.net"},"error":{"name":"Unavailable For Legal Reasons","message":""}}'

In [ ]:
labels = df.label.values

In [ ]:
vectorizer = TfidfVectorizer(max_df=0.6, min_df=1,max_features=1000,stop_words='english')

X = vectorizer.fit_transform(df['text'].values)
print("n_samples: %d, n_features: %d" % X.shape)

In [ ]:
# кластеризация
km = KMeans(n_clusters=5, init='k-means++',max_iter=100,n_init=20,random_state=1)

km.fit(X)
# метрика
print("V-measure: %0.3f" % metrics.v_measure_score(labels, km.labels_))
# выведем слова (самык ценные - часто повтор. по исполнителям)
order_centroids = kmeans.cluster_centers_.argsort()[:, ::-1]
terms = vectorizer.get_feature_names_out()
for i in range(5):
    print("Cluster %d:" % (i + 1), end='')
    for ind in order_centroids[i, :10]:
        print(' %s' % terms[ind], end='')
    print()